In [ ]:
# !pip install nibabel nilearn scikit-learn pandas numpy matplotlib
import os
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from collections import defaultdict

from nilearn.maskers import NiftiMasker
from nilearn.masking import intersect_masks
from nilearn.image import resample_to_img, load_img, math_img
from nilearn.masking import compute_multi_epi_mask

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

%matplotlib inline

## Load PsychPY timing data

In [4]:
# Convert start/stop times to scan indices
def time_to_index(times, TR=1.5):
    return np.round(np.array(times) / TR).astype(int)

In [5]:
def get_fixed_length_tr_range(start_times, TR=1.5, duration_sec=10):
    """
    Convert a list of start times to TR indices of fixed duration.
    Returns a list of lists (one TR range per event).
    """
    n_TRs = int(np.round(duration_sec / TR))  # e.g. 10s / 1.5s = 6.67 → 7
    indices = []
    for start in start_times:
        start_idx = int(np.round(start / TR))
        indices.append(list(range(start_idx, start_idx + n_TRs)))
    return indices
    
def convert_psychopy_time_to_fmri_index(beh_file, subject_id, TR=1.5):
    df = pd.read_csv(beh_file)
    picture_col = f'picture_{subject_id}'

    image_to_view_TRs = {}
    image_to_recall_TRs = {}
    imagine_category_blocks = defaultdict(list)  # 'dog' → list of TR lists, etc.

    for i, row in df.iterrows():
        image_name = row.get(picture_col)
        if pd.isna(image_name):
            continue

        # VIEW (fixed length)
        if not pd.isna(row.get('view.started')):
            tr_range = get_fixed_length_tr_range([row['view.started']], TR)[0]
            image_to_view_TRs[image_name] = tr_range

        # RECALL (fixed length)
        if not pd.isna(row.get('recall.started')):
            tr_range = get_fixed_length_tr_range([row['recall.started']], TR)[0]
            image_to_recall_TRs[image_name] = tr_range

    # IMAGINE (grouped by category)
    if 'imagine_task.started' in df.columns:
        valid_rows = df[df['imagine_task.started'].notna()]
        for _, row in valid_rows.iterrows():
            image_name = row.get(picture_col)
            if pd.isna(image_name):
                continue

            # Infer category
            if 'dog' in image_name.lower():
                category = 'dog'
            elif 'sunflower' in image_name.lower():
                category = 'sunflower'
            else:
                category = 'unknown'

            tr_range = get_fixed_length_tr_range([row['imagine_task.started']], TR)[0]
            imagine_category_blocks[category].append(tr_range)

    return {
        'view': image_to_view_TRs,                # dict: image → TRs
        'recall': image_to_recall_TRs,            # dict: image → TRs
        'imagine': dict(imagine_category_blocks)  # dict: category → list of TR lists
    }


In [13]:
# Example usage for subject 1:
subj_behavior_file = {1:"psychopy_data/1_fmri design_2025-03-04_15h31.40.655.csv",
                      3:"psychopy_data/2_fmri design_2025-03-05_15h36.18.659.csv",
                      4:"psychopy_data/4_fmri design_2025-03-05_14h28.37.417.csv"
                      }
results_sub01 = convert_psychopy_time_to_fmri_index(subj_behavior_file[1], 1, TR=1.5)
results_sub03 = convert_psychopy_time_to_fmri_index(subj_behavior_file[3], 2, TR=1.5)
results_sub04 = convert_psychopy_time_to_fmri_index(subj_behavior_file[4], 4, TR=1.5)

In [7]:
results_sub01['view']

{'dog1.jpg': [15, 16, 17, 18, 19, 20, 21],
 'dog2.jpg': [32, 33, 34, 35, 36, 37, 38],
 'dog3.jpg': [49, 50, 51, 52, 53, 54, 55],
 'dog4.jpg': [67, 68, 69, 70, 71, 72, 73],
 'dog5.jpg': [99, 100, 101, 102, 103, 104, 105],
 'dog6.jpg': [117, 118, 119, 120, 121, 122, 123],
 'dog7.jpg': [134, 135, 136, 137, 138, 139, 140],
 'dog8.jpg': [151, 152, 153, 154, 155, 156, 157],
 'dog9.jpg': [184, 185, 186, 187, 188, 189, 190],
 'dog10.jpg': [201, 202, 203, 204, 205, 206, 207],
 'dog11.jpg': [219, 220, 221, 222, 223, 224, 225],
 'dog12.jpg': [236, 237, 238, 239, 240, 241, 242],
 'sunflower1.jpg': [269, 270, 271, 272, 273, 274, 275],
 'sunflower2.jpg': [286, 287, 288, 289, 290, 291, 292],
 'sunflower3.jpg': [303, 304, 305, 306, 307, 308, 309],
 'sunflower4.jpg': [321, 322, 323, 324, 325, 326, 327],
 'sunflower5.jpg': [353, 354, 355, 356, 357, 358, 359],
 'sunflower6.jpg': [371, 372, 373, 374, 375, 376, 377],
 'sunflower7.jpg': [388, 389, 390, 391, 392, 393, 394],
 'sunflower8.jpg': [405, 406, 407,

In [8]:
results_sub01['recall']

{'dog1.jpg': [23, 24, 25, 26, 27, 28, 29],
 'dog2.jpg': [41, 42, 43, 44, 45, 46, 47],
 'dog3.jpg': [58, 59, 60, 61, 62, 63, 64],
 'dog4.jpg': [75, 76, 77, 78, 79, 80, 81],
 'dog5.jpg': [108, 109, 110, 111, 112, 113, 114],
 'dog6.jpg': [125, 126, 127, 128, 129, 130, 131],
 'dog7.jpg': [143, 144, 145, 146, 147, 148, 149],
 'dog8.jpg': [160, 161, 162, 163, 164, 165, 166],
 'dog9.jpg': [193, 194, 195, 196, 197, 198, 199],
 'dog10.jpg': [210, 211, 212, 213, 214, 215, 216],
 'dog11.jpg': [227, 228, 229, 230, 231, 232, 233],
 'dog12.jpg': [245, 246, 247, 248, 249, 250, 251],
 'sunflower1.jpg': [277, 278, 279, 280, 281, 282, 283],
 'sunflower2.jpg': [295, 296, 297, 298, 299, 300, 301],
 'sunflower3.jpg': [312, 313, 314, 315, 316, 317, 318],
 'sunflower4.jpg': [329, 330, 331, 332, 333, 334, 335],
 'sunflower5.jpg': [362, 363, 364, 365, 366, 367, 368],
 'sunflower6.jpg': [379, 380, 381, 382, 383, 384, 385],
 'sunflower7.jpg': [397, 398, 399, 400, 401, 402, 403],
 'sunflower8.jpg': [414, 415, 416

In [9]:
results_sub01['imagine']

{'dog': [[89, 90, 91, 92, 93, 94, 95],
  [174, 175, 176, 177, 178, 179, 180],
  [259, 260, 261, 262, 263, 264, 265]],
 'sunflower': [[343, 344, 345, 346, 347, 348, 349],
  [428, 429, 430, 431, 432, 433, 434],
  [512, 513, 514, 515, 516, 517, 518]]}

## Load the fMRI data

In [10]:
# Define data paths
data_dir = "/jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep" 
subjects = ["sub-01/func/", "sub-03/func/", "sub-04/func/"]  # Assuming 20 subjects, update as needed
task_prefix = "imagine"

In [105]:
output_dir = "neural_activity"
os.makedirs(output_dir, exist_ok=True)

task_prefix = "imagine"
space_tag = "space-MNI152NLin2009cAsym"  # standard space

def build_intersection_mask(subject_list, data_dir, space_tag="space-MNI152NLin2009cAsym", task_prefix="imagine"):
    mask_paths = []
    for subject in subject_list:
        subject_path = os.path.join(data_dir, subject)
        for f in os.listdir(subject_path):
            if (task_prefix in f) and (space_tag in f) and ("desc-brain_mask.nii.gz" in f):
                mask_paths.append(os.path.join(subject_path, f))
                break
    print(f"Found {len(mask_paths)} masks")
    intersected = intersect_masks(mask_paths, threshold=1.0, connected=False)
    return intersected

def extract_neural_activity(subject, group_mask_img):
    print(f"Processing {subject}...")

    # Find BOLD
    bold_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if (task_prefix in f) and (space_tag in f) and ("desc-preproc_bold.nii.gz" in f):
            bold_file = os.path.join(data_dir, subject, f)
            break

    if not bold_file:
        print(f"⚠️ No MNI-space BOLD file for {subject}")
        return None

    bold_img = nib.load(bold_file)

    # Load confounds (optional)
    confound_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if (task_prefix in f) and ("desc-confounds_timeseries.tsv" in f):
            confound_file = os.path.join(data_dir, subject, f)
            break

    confounds = None
    if confound_file:
        conf_df = pd.read_csv(confound_file, sep="\t")
        nuisance_cols = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
        nuisance_cols = [col for col in nuisance_cols if col in conf_df.columns]
        confounds = conf_df[nuisance_cols].fillna(method='bfill').fillna(method='ffill').values

    # Extract data using the shared mask
    masker = NiftiMasker(mask_img=group_mask_img, standardize=True, high_pass=0.01, t_r=1.5)
    masker.fit(bold_img)
    time_series = masker.transform(bold_img, confounds=confounds)

    print(f"Extracted neural activity shape: {time_series.shape}")
    return time_series


In [106]:
# Build shared group-level mask
group_mask = build_intersection_mask(subjects, data_dir)

# Extract aligned time series for all subjects
ts_sub01 = extract_neural_activity(subjects[0], group_mask)
ts_sub03 = extract_neural_activity(subjects[1], group_mask)
ts_sub04 = extract_neural_activity(subjects[2], group_mask)

Found 3 masks
Processing sub-01/func/...
Extracted neural activity shape: (535, 70482)
Processing sub-03/func/...
Extracted neural activity shape: (535, 70482)
Processing sub-04/func/...
Extracted neural activity shape: (535, 70482)


## Multivariate pattern analysis (whole brain analysis)
* Train the binary category classifier on the view condition
* Test it on the recall and imagine condition (also cross validation)

See notebook: fmri-4/fmri-4-mvpa-key.ipynb

Ref handbook: https://brainhack-princeton.github.io/handbook/content_pages/05-02-mvpa.html


In [67]:
def prepare_data(tr_dict, category_labels, time_series):
    """
    tr_dict: dict of image → list of TRs
    category_labels: dict of image → 0 or 1
    time_series: (T, V) matrix
    """
    X = []
    y = []
    for img, trs in tr_dict.items():
        if img not in category_labels:
            continue
        label = category_labels[img]
        for tr in trs:
            if tr < time_series.shape[0]:
                X.append(time_series[tr])
                y.append(label)
    return np.array(X), np.array(y)

def prepare_blockwise_data(tr_dict, category_labels, time_series):
    X = []
    y = []
    for img, trs in tr_dict.items():
        if img not in category_labels:
            continue
        label = category_labels[img]
        block_data = [time_series[tr] for tr in trs if tr < time_series.shape[0]]
        if block_data:
            X.append(np.mean(block_data, axis=0))  # average over TRs
            y.append(label)
    return np.array(X), np.array(y)

def prepare_imagine_data(imagine_dict, time_series, average_blocks=True):
    X, y = [], []
    for cat, blocks in imagine_dict.items():
        label = 0 if cat == 'dog' else 1
        for block in blocks:
            block_data = [time_series[tr] for tr in block if tr < time_series.shape[0]]
            if not block_data: continue
            if average_blocks:
                X.append(np.mean(block_data, axis=0))
                y.append(label)
            else:
                for tr_data in block_data:
                    X.append(tr_data)
                    y.append(label)
    return np.array(X), np.array(y)

In [94]:
def run_mvpa_pipeline_multi_subject(
    time_series_list, results_list,
    blockwise=True, apply_pca=True, model_type='logreg',
    n_components=100, average_proba_blocks=False
):
    from sklearn.linear_model import LogisticRegression, RidgeClassifier
    from sklearn.decomposition import PCA
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import cross_val_score
    from sklearn.utils import shuffle
    import numpy as np

    # === Pool training data from all subjects ===
    X_train_all, y_train_all = [], []

    for ts, results in zip(time_series_list, results_list):
        label_dict = {img: 0 if 'dog' in img else 1 for img in results['view'].keys()}
        if blockwise:
            Xi, yi = prepare_blockwise_data(results['view'], label_dict, ts)
        else:
            Xi, yi = prepare_data(results['view'], label_dict, ts)
        X_train_all.append(Xi)
        y_train_all.append(yi)

    X_train = np.concatenate(X_train_all)
    y_train = np.concatenate(y_train_all)
    X_train, y_train = shuffle(X_train, y_train, random_state=42)

    print(f"Pooled train: {X_train.shape}")

    # === PCA ===
    if apply_pca:
        pca = PCA(n_components=min(n_components, X_train.shape[0], X_train.shape[1]))
        X_train = pca.fit_transform(X_train)
    else:
        pca = None

    # === Classifier ===
    if model_type == 'logreg':
        clf = LogisticRegression(penalty='l2', solver='liblinear', C=1.0, max_iter=1000)
    elif model_type == 'ridge':
        clf = RidgeClassifier(alpha=1.0)
    else:
        raise ValueError("Unsupported model type")

    # === Train and evaluate on pooled training set ===
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5)
    print(f"CV accuracy (view): {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")
    clf.fit(X_train, y_train)
    train_acc = clf.score(X_train, y_train)
    print(f"Training accuracy (view): {train_acc:.2f}")

    # === Evaluate on recall and imagine for each subject separately ===
    for i, (ts, results) in enumerate(zip(time_series_list, results_list), start=1):
        label_dict = {img: 0 if 'dog' in img else 1 for img in results['view'].keys()}
        print(f"\n--- Testing on subject {i:02d} ---")

        # ----- Recall -----
        if average_proba_blocks:
            y_recall, recall_preds = [], []
            for img, trs in results['recall'].items():
                if img not in label_dict:
                    continue
                label = label_dict[img]
                block_data = [ts[tr] for tr in trs if tr < ts.shape[0]]
                block_array = np.array(block_data)
                if apply_pca:
                    block_array = pca.transform(block_array)
                probs = clf.predict_proba(block_array)
                avg_prob = probs.mean(axis=0)
                pred = np.argmax(avg_prob)
                y_recall.append(label)
                recall_preds.append(pred)
            acc_recall = accuracy_score(y_recall, recall_preds)
        else:
            X_recall, y_recall = prepare_data(results['recall'], label_dict, ts)
            if apply_pca:
                X_recall = pca.transform(X_recall)
            acc_recall = accuracy_score(y_recall, clf.predict(X_recall))
        print(f"Recall decoding accuracy: {acc_recall:.2f}")

        # ----- Imagine -----
        if average_proba_blocks:
            y_imagine, imagine_preds = [], []
            for cat, blocks in results['imagine'].items():
                label = 0 if cat == 'dog' else 1
                for trs in blocks:
                    block_data = [ts[tr] for tr in trs if tr < ts.shape[0]]
                    block_array = np.array(block_data)
                    if apply_pca:
                        block_array = pca.transform(block_array)
                    probs = clf.predict_proba(block_array)
                    avg_prob = probs.mean(axis=0)
                    pred = np.argmax(avg_prob)
                    y_imagine.append(label)
                    imagine_preds.append(pred)
            acc_imagine = accuracy_score(y_imagine, imagine_preds)
        else:
            X_imagine, y_imagine = prepare_imagine_data(results['imagine'], ts, average_blocks=blockwise)
            if apply_pca:
                X_imagine = pca.transform(X_imagine)
            acc_imagine = accuracy_score(y_imagine, clf.predict(X_imagine))
        print(f"Imagine decoding accuracy: {acc_imagine:.2f}")

    return clf

In [95]:
def get_category_labels(view_dict):
    return {img: 0 if 'dog' in img.lower() else 1 for img in view_dict.keys()}

In [108]:
clf = run_mvpa_pipeline_multi_subject(
    time_series_list=[ts_sub01, ts_sub03, ts_sub04],
    results_list=[results_sub01, results_sub03, results_sub04],
    blockwise=False,
    apply_pca=True,
    model_type='ridge',  # or 'ridge'
    n_components=10,
    average_proba_blocks=False  # optional smoothing
)


Pooled train: (504, 70482)
CV accuracy (view): 0.50 ± 0.03
Training accuracy (view): 0.56

--- Testing on subject 01 ---
Recall decoding accuracy: 0.49
Imagine decoding accuracy: 0.50

--- Testing on subject 02 ---
Recall decoding accuracy: 0.49
Imagine decoding accuracy: 0.52

--- Testing on subject 03 ---
Recall decoding accuracy: 0.56
Imagine decoding accuracy: 0.48


## Representational Similarity Analysis (RSA) 

* Keep the category the same (e.g. dog), what is the similarity between view, recall, and imagine conditions
* Compare the within-category similarity (within dogs vs within flowers)
* Expect flower category to be more clustered because of visual similarity
* Produce correlation matrix

See notebook: fmri-5/fmri-5-rsa-key.ipynb
